# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features
y = pd.DataFrame(boston.target,columns=['price'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

Fit the model and apply the model to the make test set predictions

In [13]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(X_train,y_train)
y_hat_train=linreg.predict(X_train)
y_hat_test=linreg.predict(X_test)

Calculate the residuals and the mean squared error

In [15]:
train_residuals=y_train-y_hat_train
test_residuals=y_test-y_hat_test

from sklearn.metrics import mean_squared_error
train_mse=mean_squared_error(y_train,y_hat_train)
test_mse=mean_squared_error(y_test,y_hat_test)

print(f"Training Mean Square Error: {train_mse}")
print(f"Testing Mean Square Error: {test_mse}")

Training Mean Square Error: 16.821201510069898
Testing Mean Square Error: 15.887117485431336


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [53]:
def kfolds_dev(data, k):
    # Force data as pandas dataframe
    data_df=pd.DataFrame(data)
    
    folds=[]
    fold_size=len(data_df.index)//k
    leftovers=len(data_df.index)%k # number of leftovers tell us how many folds need to be +1 row bigger
    start_slice=0
    for i in list(range(k)):
        if leftovers==0:
            end_slice=start_slice + fold_size-1
            print(f"Leftovers: {leftovers}, Start slice: {start_slice}, End slice: {end_slice}. Currently in 'if'.")
            folds.append(data_df.loc[start_slice:end_slice])
            start_slice=end_slice+1
        else:
            end_slice=start_slice + fold_size
            print(f"Leftovers: {leftovers}, Start slice: {start_slice}, End slice: {end_slice}. Currently in 'else'.")
            folds.append(data_df.loc[start_slice:end_slice])
            start_slice=end_slice+1
            leftovers-=1
    return folds



In [65]:
testing=[{'A':[0,1,2,3,4,5]},{'B':[0,1,2,3,4,5]},{'C':[0,1,2,3,4,5]},{'D':[0,1,2,3,4,5]},{'E':[0,1,2,3,4,5]},{'F':6}]
split=kfolds_dev(testing,4)
print(split[0])
print(split[1])
print(split[2])
print(split[3])

Leftovers: 2, Start slice: 0, End slice: 1. Currently in 'else'.
Leftovers: 1, Start slice: 2, End slice: 3. Currently in 'else'.
Leftovers: 0, Start slice: 4, End slice: 4. Currently in 'if'.
Leftovers: 0, Start slice: 5, End slice: 5. Currently in 'if'.
                    A                   B    C    D    E   F
0  [0, 1, 2, 3, 4, 5]                 NaN  NaN  NaN  NaN NaN
1                 NaN  [0, 1, 2, 3, 4, 5]  NaN  NaN  NaN NaN
     A    B                   C                   D    E   F
2  NaN  NaN  [0, 1, 2, 3, 4, 5]                 NaN  NaN NaN
3  NaN  NaN                 NaN  [0, 1, 2, 3, 4, 5]  NaN NaN
     A    B    C    D                   E   F
4  NaN  NaN  NaN  NaN  [0, 1, 2, 3, 4, 5] NaN
     A    B    C    D    E    F
5  NaN  NaN  NaN  NaN  NaN  6.0


Now that I've got it up and running correctly, I'll take out the print statements and testing stuff.

In [66]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data_df=pd.DataFrame(data)
    
    folds=[]
    fold_size=len(data_df.index)//k
    leftovers=len(data_df.index)%k # number of leftovers tell us how many folds need to be +1 row bigger
    start_slice=0
    for i in list(range(k)):
        if leftovers==0:
            end_slice=start_slice + fold_size-1
            folds.append(data_df.loc[start_slice:end_slice])
            start_slice=end_slice+1
        else:
            end_slice=start_slice + fold_size
            folds.append(data_df.loc[start_slice:end_slice])
            start_slice=end_slice+1
            leftovers-=1
    return folds

### Apply it to the Boston Housing Data

In [67]:
boston_all = pd.concat([X, y], axis=1)
boston_all.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,0.542096,1.0,296.0,15.3,1.000000,-1.275260,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,0.623954,2.0,242.0,17.8,1.000000,-0.263711,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,0.623954,2.0,242.0,17.8,0.989737,-1.627858,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,0.707895,3.0,222.0,18.7,0.994276,-2.153192,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,0.707895,3.0,222.0,18.7,1.000000,-1.162114,36.2


In [78]:
boston_splits=kfolds(boston_all,5)
for split in boston_splits:
    print(f"{split.index+1}: Length {len(split)}")

RangeIndex(start=1, stop=103, step=1): Length 102
RangeIndex(start=103, stop=204, step=1): Length 101
RangeIndex(start=204, stop=305, step=1): Length 101
RangeIndex(start=305, stop=406, step=1): Length 101
RangeIndex(start=406, stop=507, step=1): Length 101


### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [98]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(boston_splits) if i!=n])
    test = boston_splits[n]
    
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(train[X.columns],train[y.columns])
    
    #predict prices
    y_hat_train=linreg.predict(train[X.columns])
    y_hat_test=linreg.predict(test[X.columns])
    
    #Evaluate Train and Test Errors
    train_error=mean_squared_error(train[y.columns],y_hat_train)
    test_error=mean_squared_error(test[y.columns],y_hat_test)
    
    train_errs.append(train_error)
    test_errs.append(test_error)

print(train_errs)
print(test_errs)

[17.9185670542463, 17.3577081046629, 15.545678258525871, 11.03762238964458, 17.23404426556592]
[13.016192102045736, 14.628321831424671, 24.81432997168208, 55.24107772637733, 19.022337999169462]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [101]:
from sklearn.model_selection import cross_val_score
five_fold_val_scores=cross_val_score(linreg,X,y,cv=5, scoring="neg_mean_squared_error")

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [105]:
print(five_fold_val_scores.mean())
print(np.array(test_errs).mean())

-25.344451926139904
25.344451926139858


##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!